In [1]:
import pandas as pd
import numpy as np
from sklearn import cross_validation, svm, preprocessing, metrics
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## 建Label dict
- key是FileID
- value是Label
- 方便最後塞Label用

In [2]:
df_train = pd.DataFrame(pd.read_csv('training-set.csv',header=None))
df_train.columns = ['FileID','Label']

dic = df_train.set_index('FileID').to_dict()

dic = dic['Label']

## 讀檔
1. 把之前merge好的大csv讀進來
2. Unnamed: 0欄位是沒用的，先刪掉
3. 增加一個count欄位，方便算比數（應該有更好的寫法）
4. 根據FileID group 幾個欄位出來
    - 'Cnum':每個FileID有對應到的不同的CustomerID
    - 'Pnum':每個FileID有對應到的不同的ProductID
    - 'df_guby_t_max':每個FileID出現的最後時間
    - 'df_guby_t_min':每個FileID第一次出現的時間
    - 'times':每個FileID出現的次數
    - 'time_diff'= df_guby_t_max - df_guby_t_min存在多長的時間，以天為單位
    - 'tzone1' :檔案在0~6點被開啟的比例
    - 'tzone2' :檔案在7~12點被開啟的比例
    - 'tzone3' :檔案在13~18點被開啟的比例
    - 'tzone4' :檔案在19~0點被開啟的比例
5. 把'df_guby_t_max'及'df_guby_t_min'兩個欄位刪掉（比較沒有意義）
6. 用上面建立好的dict把Label併進來
7. 把Label != 3的歸類為X(train data);Label == 3 的歸類為y(test data)
8. 用train_test_split把X分成訓練跟驗證資料

In [3]:
df_log = pd.DataFrame(pd.read_csv('total_log.csv'))

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
df_log.tail()

,FileID,CustomerID,QueryTS,ProductID,Dtime,Label,count
83273105,965a5b2bfd7b11f8399c1c0bf4363748,f54adb48251c96cb369aaf77cc23a7d7,1496275182,55649,2017-05-31 23:59:42,3,1
83273106,2b079292f52388ef1d6d4fa1330d6f7e,286b944e21a255f44b28b4a2ce5f9f54,1496275185,55649,2017-05-31 23:59:45,3,1
83273107,2f4e50c843bfb8c3c691f892b7ba5b30,4f564b23a24fac46fcd8deaa80b66e52,1496275190,55649,2017-05-31 23:59:50,3,1
83273108,ab84a6314d8f27d0f5fac69a00979cfb,fdf59bed2c8756db5585dce51e6933d9,1496275191,55649,2017-05-31 23:59:51,3,1
83273109,9f68e6f9c56e3d26d65b18f0ae2268c8,7220b13b2f1ec5a8b0094323ad888ae2,1496275198,55649,2017-05-31 23:59:58,3,1


In [29]:
int(df_log['Dtime'][0][11:-6])

0

In [5]:
df_log = df_log.drop('Unnamed: 0',axis = 1)

In [6]:
df_log['count'] = 1

In [44]:
df_log['apm'] = pd.to_numeric(df_log['Dtime'].str[11:-6])

In [65]:
z1 = list(df_log[(df_log['apm']>0) & (df_log['apm']<=6)].index.values)
z2 = list(df_log[(df_log['apm']>6) & (df_log['apm']<=12)].index.values)
z3 = list(df_log[(df_log['apm']>12) & (df_log['apm']<=18)].index.values)
z4 = list(df_log[(df_log['apm']>18) | (df_log['apm']==0)].index.values)

In [85]:
df_log['tzone1'] = 0
df_log['tzone2'] = 0
df_log['tzone3'] = 0
df_log['tzone4'] = 0 
df_log.loc[z1,'tzone1'] = 1
df_log.loc[z2,'tzone2'] = 1
df_log.loc[z3,'tzone3'] = 1
df_log.loc[z4,'tzone4'] = 1

In [89]:
df_tz = df_log.groupby(['FileID'])['tzone1','tzone2','tzone3','tzone4'].sum()

In [93]:
df_tz.max()

tzone1    39697
tzone2    46133
tzone3    26155
tzone4    19949
dtype: int64

In [7]:
df_guby_t_max = df_log.groupby('FileID')['QueryTS'].max()
df_guby_t_min = df_log.groupby('FileID')['QueryTS'].min()

In [12]:
df_guby = df_log.groupby('FileID').CustomerID.nunique()

In [13]:
df_gubyp = df_log.groupby('FileID').ProductID.nunique()

In [15]:
df_gubycoun = df_log.groupby('FileID')['count'].sum()

In [16]:
total_data = pd.concat([df_guby, df_gubyp, df_guby_t_max, df_guby_t_min, df_gubycoun, df_tz], axis=1)
total_data.columns = ['Cnum','Pnum','df_guby_t_max','df_guby_t_min','times','tzone1','tzone2','tzone3','tzone4']

In [17]:
total_data['time_diff'] = (total_data['df_guby_t_max'] - total_data['df_guby_t_min'])/(60*60*24)

In [19]:
total_data = total_data.drop(['df_guby_t_max', 'df_guby_t_min'],axis = 1)

In [97]:
total_data['tzone1'] = (total_data['tzone1'] / total_data['times'])*100
total_data['tzone2'] = (total_data['tzone2'] / total_data['times'])*100 
total_data['tzone3'] = (total_data['tzone3'] / total_data['times'])*100 
total_data['tzone4'] = (total_data['tzone4'] / total_data['times'])*100

In [103]:
ta = total_data.drop('Label',axis=1)

In [104]:
ta.to_csv('feature_byYoga.csv', sep=',', encoding='utf-8')

In [32]:
total_data['Label'] = 3
for i in range(len(total_data)):
    try:
        total_data['Label'][i] = dic[total_data.index[i]]
    except:
        total_data['Label'][i] = 3

NameError: name 'train_data' is not defined

In [72]:
X = train_data[train_data['Label'] != 3]
y = train_data[train_data['Label'] == 3]

In [76]:
X = X.drop('Label',axis = 1)

In [82]:
df_train = df_train.set_index('FileID')

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, df_train, random_state = 7, test_size = 0.25)

In [ ]:
# x = train_data.values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# df = pd.DataFrame(x_scaled)

## GBM
- 裝XGB的時候碰到了問題，所以先用GBM做了
- 做出來的預測是0,1不是機率（有點大的問題）
- train的時候沒有做很多的校調，一次就傳上去了（結果只有0.5哈哈哈哈（別打我

In [94]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [90]:
clf = GradientBoostingClassifier(n_estimators=500,max_depth=10,min_samples_split=5,min_samples_leaf=3)

In [91]:
clf.fit(X_train,y_train)

/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=10,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=3, min_samples_split=5,
              min_weight_fraction_leaf=0.0, n_estimators=500,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [92]:
pred_y = clf.predict(X_test)

In [95]:
accuracy = accuracy_score(y_test, pred_y )
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 90.99%


In [101]:
y_pred = clf.predict(y)

In [100]:
y = y.drop('Label',axis = 1)

In [105]:
y['Probability'] = y_pred

In [112]:
y_final = y.drop(['Cnum','Pnum','times','time_diff'],axis = 1)

In [115]:
y_final['FileID'] = y_final.index

In [119]:
df_y = y_final.reset_index(drop=True)

In [121]:
df_y = df_y[['FileID', 'Probability']]

In [123]:
df_y.to_csv('tbrain_v2.csv', sep=',', encoding='utf-8')